In [1]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 

In [2]:
# notebooks/example_notebook.ipynb

import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [3]:
from load_data import load_data_from_postgres, load_data_using_sqlalchemy
from sql_queries import execute_telecom_queries


In [4]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Fetch database connection parameters from environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [5]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
df = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")


c:\Users\hp\kifiya_10_acadamy\kifiya_week2_assignment\scripts\load_data.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded the data


In [6]:


df['TCP Retransmission'] = df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']
df['RTT'] = df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)'] 
df['Throughput'] = df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']

# Handle missing values
df["TCP Retransmission"].fillna(df["TCP Retransmission"].mean(), inplace=True)
df["RTT"].fillna(df["RTT"].mean(), inplace=True)
df["Handset Type"].fillna(df["Handset Type"].mode()[0], inplace=True)
df["Throughput"].fillna(df["Throughput"].mean(), inplace=True)

# Handle outliers using IQR method
for col in ["TCP Retransmission", "RTT", "Throughput"]:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Replace outliers with mean
    df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), df[col].mean(), df[col])

# Aggregate metrics per customer
aggregated_df = df.groupby("MSISDN/Number").agg(
    Avg_TCP_Retransmission=("TCP Retransmission", "mean"),
    Avg_RTT=("RTT", "mean"),
    Handset_Type=("Handset Type", lambda x: x.mode()[0]),
    Avg_Throughput=("Throughput", "mean")
).reset_index()

# Output the aggregated DataFrame
print(aggregated_df)


C:\Users\hp\AppData\Local\Temp\ipykernel_4892\815334861.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["TCP Retransmission"].fillna(df["TCP Retransmission"].mean(), inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4892\815334861.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

        MSISDN/Number  Avg_TCP_Retransmission     Avg_RTT  \
0        3.360100e+10            2.571697e+07   46.000000   
1        3.360100e+10            2.571697e+07   31.000000   
2        3.360100e+10            2.571697e+07  127.024885   
3        3.360101e+10            2.571697e+07   84.000000   
4        3.360101e+10            1.754390e+07   59.500000   
...               ...                     ...         ...   
106851   3.379000e+10            2.180450e+05   52.000000   
106852   3.379000e+10            2.571697e+07   40.000000   
106853   3.197021e+12            2.571697e+07  127.024885   
106854   3.370000e+14            2.571697e+07  127.024885   
106855   8.823971e+14            2.571697e+07  127.024885   

                            Handset_Type  Avg_Throughput  
0         Huawei P20 Lite Huawei Nova 3E       76.000000  
1                 Apple iPhone 7 (A1778)       99.000000  
2                              undefined       97.000000  
3                Apple iPhone 5

In [12]:
# Compute top 10, bottom 10, and most frequent values for specified metrics

# Function to compute top, bottom, and most frequent values
def compute_statistics(column, name):
    top_10 = column.nlargest(10).values
    bottom_10 = column.nsmallest(10).values
    most_frequent = column.value_counts().head(10).index.values
    return {
        f"Top 10 {name}": top_10,
        f"Bottom 10 {name}": bottom_10,
        f"Most Frequent {name}": most_frequent,
    }

# Metrics to compute statistics for
metrics = {
    "TCP Retransmission": df["TCP Retransmission"],
    "RTT": df["RTT"],
    "Throughput": df["Throughput"],
}

# Compute statistics for each metric
statistics = {}
for metric_name, metric_data in metrics.items():
    statistics[metric_name] = compute_statistics(metric_data, metric_name)

# Display results
for metric, stats in statistics.items():
    print(f"\n--- {metric} ---")
    for stat_name, values in stats.items():
        print(f"{stat_name}: {values}")



--- TCP Retransmission ---
Top 10 TCP Retransmission: [56212033. 56202254. 56199084. 56195532. 56185212. 56176107. 56164100.
 56148826. 56144622. 56142569.]
Bottom 10 TCP Retransmission: [ 86.  97. 106. 108. 113. 128. 129. 134. 134. 143.]
Most Frequent TCP Retransmission: [2.57169663e+07 2.57169663e+07 2.66000000e+03 5.32000000e+03
 6.65000000e+03 3.99000000e+03 7.98000000e+03 1.87300000e+03
 2.64800000e+03 1.59600000e+04]

--- RTT ---
Top 10 RTT: [257. 257. 257. 257. 257. 257. 257. 257. 257. 257.]
Bottom 10 RTT: [0. 0. 0. 0. 2. 4. 4. 5. 6. 6.]
Most Frequent RTT: [127.02488519 127.02488519  29.          39.          38.
  40.          30.          28.          49.          41.        ]

--- Throughput ---
Top 10 Throughput: [55732. 55732. 55731. 55726. 55724. 55721. 55718. 55715. 55713. 55708.]
Bottom 10 Throughput: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Most Frequent Throughput: [1.50704746e+04 6.30000000e+01 1.50000000e+01 9.70000000e+01
 9.00000000e+01 9.80000000e+01 9.60000000e+01 9.9000

In [16]:

# Select relevant metrics
metrics = df[["TCP Retransmission", "RTT", "Throughput"]]

# Scale the metrics
scaler = StandardScaler()
metrics_scaled = scaler.fit_transform(metrics)

# Run k-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df["Cluster"] = kmeans.fit_predict(metrics_scaled)

# Profile each cluster
cluster_profile = df.groupby("Cluster").agg(
    Avg_TCP_Retransmission=("TCP Retransmission", "mean"),
    Avg_RTT=("RTT", "mean"),
    Avg_Throughput=("Throughput", "mean"),
    User_Count=("MSISDN/Number", "count"),
).reset_index()

# Print results
print("Cluster Profiles:\n", cluster_profile)


Cluster Profiles:
    Cluster  Avg_TCP_Retransmission     Avg_RTT  Avg_Throughput  User_Count
0        0            2.571341e+07   42.804134     1979.270877       65038
1        1            3.584048e+06   92.523105    24484.477197       45048
2        2            2.547753e+07  131.686389     2890.301898       38849
